In [ ]:
!pip install openai-whisper  pydub



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=2d02602a45e2a73c3b6f6aa6e99db7ccd75cdc995567a75e8993aaed7498731f
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
!pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
from pydub import AudioSegment
import whisper
import numpy as np
import torch
from IPython.display import Audio, display
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import csv
import random


In [ ]:
!kaggle datasets download imsparsh/audio-speech-sentiment

Dataset URL: https://www.kaggle.com/datasets/imsparsh/audio-speech-sentiment
License(s): CC0-1.0
 99% 230M/231M [00:04<00:00, 104MB/s] 
100% 231M/231M [00:04<00:00, 51.0MB/s]


In [ ]:
import zipfile

# Unzip the dataset
with zipfile.ZipFile('/content/audio-speech-sentiment.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')


In [ ]:
def Speech2Text(audio_path):


    # Function to perform speech-to-text
    def transcribe_audio(model, audio_path: str) -> str:
        # Load the audio


        # Run the model
        result = model.transcribe(audio_path)

        return result['text']

    # Load the Whisper model
    model = whisper.load_model("small", device="cuda")
    audio_file_path = audio_path

    transcription = transcribe_audio(model, audio_file_path)
    return transcription

In [ ]:
def transcribe_wav_files(wav_dir):


    with open("transcription_dataset.csv", "a", newline='') as csvfile:
        fieldnames = ['filename', 'transcription']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if csvfile.tell() == 0:  # Check if file is empty
            writer.writeheader()

        for filename in tqdm(os.listdir(wav_dir)):
            if filename.endswith(".wav"):
                wav_file_path = os.path.join(wav_dir, filename)
                text = Speech2Text(wav_file_path)
                writer.writerow({'filename': filename, 'transcription': text})
                print(f"Transcription of {filename}: {text}")

In [ ]:
wav_directory = "/content/TRAIN"
transcribe_wav_files(wav_directory)

  0%|          | 0/250 [00:00<?, ?it/s]
  0%|                                               | 0.00/461M [00:00<?, ?iB/s]
  1%|▍                                     | 4.83M/461M [00:00<00:09, 50.6MiB/s]
  3%|█▏                                    | 13.7M/461M [00:00<00:06, 73.5MiB/s]
  4%|█▋                                    | 20.7M/461M [00:00<00:06, 70.7MiB/s]
  6%|██▍                                   | 29.0M/461M [00:00<00:05, 77.2MiB/s]
  8%|███                                   | 36.4M/461M [00:00<00:07, 63.2MiB/s]
 10%|███▋                                  | 45.3M/461M [00:00<00:06, 69.1MiB/s]
 12%|████▍                                 | 53.3M/461M [00:00<00:05, 73.3MiB/s]
 15%|█████▌                                | 68.2M/461M [00:00<00:04, 97.4MiB/s]
 17%|██████▍                               | 77.9M/461M [00:01<00:04, 96.2MiB/s]
 19%|███████▏                              | 87.3M/461M [00:01<00:04, 96.0MiB/s]
 21%|███████▉                              | 96.7M/461M [00:01<00:05,

Transcription of 71.wav:  I believe in you, I love you.


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
  1%|        

Transcription of 190.wav:  You should explain more.


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
  1%|        

...

Transcription of 235.wav:  You should learn more good manners.


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
100%|████████

Transcription of 137.wav:  That sounds great but please tell me more on this.


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
100%|████████

Transcription of 353.wav:  you have noticed the擊��ำ c


In [ ]:
# Load datasets
transcriptions = pd.read_csv("/content/transcription_dataset.csv")  # Contains filename, transcription
transcriptions.columns = ['Filename', 'transcription']
sentiments = pd.read_csv("/content/TRAIN.csv")  # Contains filename, sentiment



In [ ]:
transcriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Filename       250 non-null    object
 1   transcription  250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [ ]:
# Merge datasets on 'filename'
data = pd.merge(transcriptions, sentiments, on="Filename")


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Filename       250 non-null    object
 1   transcription  250 non-null    object
 2   Class          250 non-null    object
dtypes: object(3)
memory usage: 6.0+ KB


In [ ]:
# Preprocess the data
data.dropna(inplace=True)  # Drop rows with missing values
label_encoder = LabelEncoder()
data["sentiment_label"] = label_encoder.fit_transform(data["Class"])


In [ ]:
data.to_csv("sentiment_data.csv", index=False)

In [ ]:
# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["transcription"].tolist(),
    data["sentiment_label"].tolist(),
    test_size=0.2,
    random_state=42
)


In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"],
                                   "attention_mask": train_encodings["attention_mask"],
                                   "labels": train_labels})
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"],
                                  "attention_mask": test_encodings["attention_mask"],
                                  "labels": test_labels})


In [ ]:
# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # Enable logging to Weights & Biases
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()



Epoch,Training Loss,Validation Loss
1,1.075100,0.898639
2,0.941500,0.770397
3,0.773900,0.660438
4,0.598500,0.630065
5,0.540300,0.629252


TrainOutput(global_step=65, training_loss=0.7526580333709717, metrics={'train_runtime': 47.2646, 'train_samples_per_second': 21.157, 'train_steps_per_second': 1.375, 'total_flos': 14389015032000.0, 'train_loss': 0.7526580333709717, 'epoch': 5.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

('./sentiment_model/tokenizer_config.json',
 './sentiment_model/special_tokens_map.json',
 './sentiment_model/vocab.txt',
 './sentiment_model/added_tokens.json')

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6292515397071838, 'eval_runtime': 0.1364, 'eval_samples_per_second': 366.52, 'eval_steps_per_second': 29.322, 'epoch': 5.0}


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Move inputs to the model's device
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)
    return label_encoder.inverse_transform(predictions.tolist())

print(predict_sentiment("The product is bad."))

['Negative']


In [ ]:
# prompt: # Compare predictions with ground truth (if available)

# Assuming 'test_labels' contains the ground truth labels and 'trainer' is your trained model.
# Get predictions on the test set
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Compare predictions with ground truth
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

print(classification_report(test_labels, predicted_labels, target_names=label_encoder.classes_))

Accuracy: 0.8
              precision    recall  f1-score   support

    Negative       0.77      0.62      0.69        16
     Neutral       0.95      0.95      0.95        19
    Positive       0.67      0.80      0.73        15

    accuracy                           0.80        50
   macro avg       0.79      0.79      0.79        50
weighted avg       0.81      0.80      0.80        50



In [ ]:
def select_random_file(directory_path):
    try:
        files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
        if not files:
            return None
        return os.path.join(directory_path, random.choice(files))
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

directory = "TEST/"


In [ ]:
random_file = select_random_file(directory)
print(random_file)
result = Speech2Text(random_file)
display(Audio(random_file))
print(result,predict_sentiment(result))

TEST/312.wav


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


 You are fired. ['Negative']
